# Packet Parsing - Index

In [191]:
import pandas as pd
import pdfplumber
import re

In [192]:
comms_index_pg_start = 0
comms_index_pg_end = 1
comms_fpath = "./pubcomm/raw_pdfs/7_11_2023; CLK - Communication; Public; ; REGULAR; Supplemental Communicatons 2.pdf"

In [183]:
pdf = pdfplumber.open(comms_fpath)

In [184]:
### Use pdfplumber or OCR
### Index sometimes can be plumbed

In [185]:
index_pages = pdf.pages[comms_index_pg_start: comms_index_pg_end + 1]
index_pages

[<Page:1>, <Page:2>]

In [174]:
for page in index_pages:
    print(page.extract_text())

In [175]:
index_pages[1].extract_text()

''

In [186]:
import os
import pytesseract

from pdf2image import convert_from_path
from PIL import Image

In [187]:
pages = convert_from_path(comms_fpath, 350)

In [199]:
pdf_folder = "./pubcomm/pdf_images_2b/"

In [196]:
# CAN SKIP - ALREADY SAVED DURING CONTENT PARSING
# for idx, page in enumerate(pages[comms_index_pg_start: comms_index_pg_end + 1]):
#     # Save pages as images in the pdf
#     page.save(index_pdf_folder + 'page'+ str(idx) +'.jpg', 'JPEG')

In [200]:
def get_text_from_pg_img(img_path):
    return pytesseract.image_to_string(Image.open(img_path))

def get_all_files(dir_path):
    """
    Lists all files in specified directory.

    `dir_path`: path to target directory
    """
    dir_list = os.listdir(dir_path)

    print("Files and directories in '", dir_path, "' :")
    print(dir_list)
    return sorted(dir_list)

In [201]:
with open("./pubcomm/index_set_2.txt", "w") as f:
    for img_path_idx in range(comms_index_pg_start, comms_index_pg_end + 1):
        img_path = f"{pdf_folder}page{img_path_idx}.jpg"
        page_text = get_text_from_pg_img(img_path)
        f.write(page_text)
        f.write("\n----\n")

In [202]:
with open("./pubcomm/index_set_2.txt", "r") as f:
    text = f.read()

In [272]:
items_speakers_list = [item.strip().split("\n") for item in text.split("Item ")]
items_speakers_list

[['SUPPLEMENTAL',
  'COMMUNICATIONS AND',
  'REPORTS 2',
  '',
  'BERKELEY CITY COUNCIL',
  'REGULAR MEETING |',
  '',
  'DATE OF MEETING: TUESDAY, JULY 11, 2023',
  'TIME: 6:00 P.M.',
  '',
  'The agenda packet for this meeting was distributed/posted on June 29, 2023. Communications in this',
  'supplement were received after 5pm on July 5, 2023. This communication packet was distributed/posted on',
  'July 10, 2023.',
  '',
  'Consent Calendar',
  '',
  'Each item in this supplement follows the corresponding item on the ay Council Agenda',
  'for this date.'],
 ['#29: Resolution Opposing Tokyo Electric Power Company and the',
  'Government of Japan’s Planned Discharge of Wastewater from Fukushima',
  'Daiichi Nuclear Power Plant into the Pacific Ocean',
  '',
  '131. Seaver Wang, on behalf of The Breakthrough Institute',
  '',
  '132. Diane Bohn',
  '',
  '133. Miya Sommers',
  '',
  'Action Calendar — New Business'],
 ['#31: Staff Shortages: City Services Constrained by Staff Retent

In [283]:
item_speakers_dict_list = []
for item_speakers in items_speakers_list[1:]:
    item = ""
    for line in item_speakers:
        m = re.search(r"([0-9]+)\.\s*([-A-Za-z]+\s*[A-Z]*)", line, re.IGNORECASE)
        if not m:
            item = f"{item} {line}"
        else:
            name = m.group(2)
            if name not in ["Supplemental material", "Revised material"]:
                try:
                    item_speaker_obj = {
                        "item_num": re.search(r"#([0-9]+):", item).group(1),
                        "item_subject": re.search(r"#[0-9]+:\s*(.*\s*.*)", item).group(1).strip(),
                        "comms_num": m.group(1),
                        "name": m.group(2)
                    }
                    item_speakers_dict_list.append(item_speaker_obj)
                except:
                    print(item)
item_speakers_dict_list

[{'item_num': '29',
  'item_subject': 'Resolution Opposing Tokyo Electric Power Company and the Government of Japan’s Planned Discharge of Wastewater from Fukushima Daiichi Nuclear Power Plant into the Pacific Ocean',
  'comms_num': '131',
  'name': 'Seaver Wang'},
 {'item_num': '29',
  'item_subject': 'Resolution Opposing Tokyo Electric Power Company and the Government of Japan’s Planned Discharge of Wastewater from Fukushima Daiichi Nuclear Power Plant into the Pacific Ocean',
  'comms_num': '132',
  'name': 'Diane Bohn'},
 {'item_num': '29',
  'item_subject': 'Resolution Opposing Tokyo Electric Power Company and the Government of Japan’s Planned Discharge of Wastewater from Fukushima Daiichi Nuclear Power Plant into the Pacific Ocean',
  'comms_num': '133',
  'name': 'Miya Sommers'},
 {'item_num': '32',
  'item_subject': 'Referral Response: Affordable Housing Preference Policy for Rental Housing Created Through the Below Market Rate and Housing Trust Fund Programs',
  'comms_num': '

In [280]:
item_speakers_df = pd.DataFrame(item_speakers_dict_list)

In [281]:
item_speakers_df

,item_num,item_subject,comms_num,name
0,29,Resolution Opposing Tokyo Electric Power Compa...,131,Seaver Wang
1,29,Resolution Opposing Tokyo Electric Power Compa...,132,Diane Bohn
2,29,Resolution Opposing Tokyo Electric Power Compa...,133,Miya Sommers
3,32,Referral Response: Affordable Housing Preferen...,135,Carole Marasovic
4,33,Adoption — Civic Center Phase II — Design Concept,136,Chase Hommeyer
5,33,Adoption — Civic Center Phase II — Design Concept,137,Michael Weber
6,33,Adoption — Civic Center Phase II — Design Concept,138,Miranda Ewell
7,33,Adoption — Civic Center Phase II — Design Concept,139,Ruth Morgan
8,33,Adoption — Civic Center Phase II — Design Conc...,140,Summer Brenner
9,33,Adoption — Civic Center Phase II — Design Conc...,141,Drew Goetting


In [282]:
item_speakers_df.to_csv("./pubcomm/item_speakers_set_2.csv")